# Tutorial: LangChain - Models, Prompts, and Output Parsers

This tutorial provides a comprehensive guide to using **LangChain**, a framework that simplifies the development of applications powered by large language models (LLMs)[2][3][4].

In this notebook, you will learn the following:

- How to set up your environment (installing required packages and setting API keys).
- How to perform direct API calls to OpenAI using a custom function.
- How to leverage LangChain’s higher-level abstractions to build chat models with prompt templates.
- How to use output parsers to convert raw LLM text responses into structured JSON data.

LangChain is a modular framework that allows developers to combine language models with external data sources and processing pipelines efficiently, saving time and reducing complexity in application development.

## Environment Setup

Before you begin, make sure you have the required packages installed and that your OpenAI API key is properly configured. The following steps load environment variables from a `.env` file and install packages if needed.

In [ ]:
# Uncomment the following lines to install required packages if they are not already installed.
# !pip install python-dotenv
# !pip install openai
# !pip install langchain

In [ ]:
# Import necessary libraries and load environment variables
import os
import openai
from dotenv import load_dotenv, find_dotenv

# This loads variables from your local .env file into the environment
_ = load_dotenv(find_dotenv())

# Set the OpenAI API key from the environment variable. Make sure your .env file contains the OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']

## Direct API Call to OpenAI

The following section defines a function `get_completion` that makes a direct API call to OpenAI’s **ChatCompletion** endpoint. This function takes a `prompt` and optionally a `model` argument. It uses a deterministic `temperature` (set to 0) so that the output remains consistent every time.

The model usage is controlled by the current date. If the date is past a specified target date, it uses the latest model version ("gpt-3.5-turbo"); otherwise, it uses an older snapshot ("gpt-3.5-turbo-0301").

In [ ]:
import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Specify the target date for updating the model
target_date = datetime.date(2024, 6, 12)

# Select the model based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

def get_completion(prompt, model=llm_model):
    """
    Send a prompt to the OpenAI ChatCompletion API and return the model's response.

    Arguments:
      prompt (str): The text prompt to send to the model.
      model (str): The name of the model to use (default is controlled by the current date).
    """
    # Structure the message as required by the API: role and content
    messages = [{"role": "user", "content": prompt}]
    
    # Call OpenAI's ChatCompletion API with specified parameters
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # Lower temperature for deterministic responses
    )
    
    # Return the text content of the first response
    return response.choices[0].message["content"]

# Example of using the get_completion function
print(get_completion("What is 1+1?"))  # Expected output is '2' or a similar deterministic answer

## Using LangChain's Chat API

LangChain provides abstraction layers that make it easier to work with language models. In this section, we use the `ChatOpenAI` class from LangChain to create a chat model. By doing so, you don’t have to manage the low-level API details directly.

We also demonstrate how to build a prompt template with placeholders (e.g., `{style}` and `{text}`) using the `ChatPromptTemplate` class. This template can dynamically format messages before sending them to the language model.

In [ ]:
from langchain.chat_models import ChatOpenAI

# Create a ChatOpenAI instance with no randomness (temperature=0.0) and the chosen model
chat = ChatOpenAI(temperature=0.0, model=llm_model)
print(chat)  # This displays information about the chat model instance

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Define a prompt template that instructs the model to translate text into a specific style
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ``````
"""

# Create the prompt template object from the template string
prompt_template = ChatPromptTemplate.from_template(template_string)

# Verify the template's content to ensure it is set up correctly
print(prompt_template.messages[0].prompt)

In [ ]:
# Define sample input text (in this case, a customer email written in a pirate style)
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# Define the target style for translation (American English, calm and respectful)
customer_style = """
American English \
in a calm and respectful tone
"""

# Format the prompt message by inserting the customer's email and desired style into the template
customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email
)

print(type(customer_messages))  # Should output a list type
print(customer_messages[0])        # Displays the formatted message

# Call the chat model with the formatted prompt and print the translated response
customer_response = chat(customer_messages)
print(customer_response.content)

## Output Parsers in LangChain

Output parsers are used to convert raw textual responses from an LLM into structured data such as JSON, which is helpful when expecting specific output formats.

The following code uses LangChain’s `ResponseSchema` and `StructuredOutputParser` classes. It defines a response schema with three keys:

- **gift**: Indicates if an item was purchased as a gift.
- **delivery_days**: The number of days it took for delivery (or -1 if unknown).
- **price_value**: Sentences about the pricing or value of an item, returned as a comma-separated list.

After defining the schemas, the notebook creates a format instruction string which is then appended to a prompt template. The final response from the LLM is parsed into a Python dictionary for easy use.

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# Define the expected structured output schemas
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown."
)

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this information is not found, output -1."
)

price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, and output them as a comma separated Python list."
)

# Combine the schemas into a list
response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

# Build a StructuredOutputParser from the response schemas
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get instructions on how to format the output (this will be appended to the prompt for guidance)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

In [ ]:
# Define an example review text that describes product features and delivery details
customer_review = """
This leaf blower is pretty amazing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# Create a review prompt template using the expected output instructions
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

from langchain.prompts import ChatPromptTemplate

# Create a ChatPromptTemplate using the review template
prompt = ChatPromptTemplate.from_template(template=review_template)

# Format the message with the review text and the format instructions
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

# Call the chat model to obtain and print the raw response
response = chat(messages)
print(response.content)

# Parse the response into a structured Python dictionary
output_dict = output_parser.parse(response.content)

# Print the parsed output
print(output_dict)

## Conclusion

In this tutorial, we have demonstrated:

- **Direct API calls** to OpenAI to generate text responses.
- Using **LangChain’s Chat API** to create chat models and format dynamic prompts.
- Leveraging the **Prompt Template** module to insert variables into text templates.
- Utilizing **Output Parsers** to convert raw text responses into structured JSON data.

LangChain is a valuable toolkit for AI developers because it abstracts many of the complexities involved in building, testing, and deploying language model applications.

Happy coding and exploration with LangChain!